# 舵机小车跟随目标

### 1、objectId 是要跟随的目标，1是人

In [1]:
objectId = 1#数字代表要跟随的目标,37是球

### 2、先加载检测目标的模型

In [2]:
from jetbot import ObjectDetector
objects_and_index ={0:'background背景',1:'person人',2:'bicycle自行车',3:'car小汽车',4:'motorcycle摩托车',5:'airplane飞机',6:'bus公共汽车',7:'train火车',8:'truck卡车',9:'boat船',10:'traffic light交通灯',11:'fire hydrant消防栓',12:'12',13:'stop sign停车标志',14:'parking meter停车收费表',15:'bench长凳',16:'bird鸟',17:'cat猫',18:'dog狗',19:'horse马',20:'sheep羊',21:'cow牛',22:'elephant大象',23:'bear熊',24:'zebra斑马',25:'giraffe长颈鹿',26:'26',27:'backpack背包',28:'umbrella雨伞',29:'29',30:'30',31:'handbag手提包',32:'tie领带',33:'suitcase手提箱',34:'frisbee飞盘',35:'skis滑雪板',36:'snowboard滑雪板',37:'sports ball运动球',38:'kite风筝',39:'baseball bat棒球棒',40:'baseball glove棒球手套',41:'skateboard滑板',42:'surfboard冲浪板',43:'tennis racket网球拍',44:'bottle瓶子',45:'45',46:'wine glass酒杯',47:'cup杯子',48:'fork叉',49:'knife刀',50:'spoon勺子',51:'bowl碗',52:'banana香蕉',53:'apple苹果',54:'sandwich三明治',55:'orange橙子;柑橘',56:'broccoli绿菜花;西兰花',57:'carrot胡萝卜',58:'hot dog热狗(香肠面包)',59:'pizza比萨饼;意大利饼',60:'donut油炸圈饼；圆环图；甜甜圈',61:'cake糕饼;蛋糕',62:'chair椅子',63:'couch长沙发',64:'potted plant盆栽',65:'bed床',66:'66',67:'dining table餐桌',68:'68',69:'69',70:'toilet坐便器',71:'71',72:'tv',73:'laptop笔记本电脑',74:'mouse鼠标',75:'remote遥控',76:'keyboard键盘',77:'cell phone手机;移动电话',78:'microwave微波炉',79:'oven烤箱;烤炉',80:'toaster(电的)烤面包片',81:'sink洗涤池,洗碗槽',82:'refrigerator(电)冰箱',83:'83',84:'book书',85:'clock时钟;钟',86:'vase花瓶;装饰瓶',87:'scissors剪刀',88:'teddy bear泰迪熊',89:'hair drier吹风机;电吹风',90:'toothbrush牙刷'}
model = ObjectDetector('ssd_mobilenet_v2_coco.engine')

### 3、引入python的常用函数库

In [3]:
import cv2
import sys
import time
import numpy as np
from datetime import datetime
from IPython.display import display
import ipywidgets.widgets as widgets
from IPython.display import clear_output

### 4、引入自定义函数库

In [4]:
from jetbot import Pwm
from jetbot import Robot
from jetbot import Camera
from jetbot import bgr8_to_jpeg

### 5、计数相关的全局变量，避免每帧都调用一次

In [5]:
t_turn1 = 0
robot = Robot() #用来操控机器人小车

### 6、自定义方法用来显示检测到的目标

In [6]:
def print_names(detections):
    s = ''
    for x in detections[0]:
        idx = x.get('label')
        confidence = x.get('confidence')
        name = objects_and_index.get(idx)
        s = s +'看到了物品：'+ name+', 确信度是: '+str(confidence) +'\n'
    clear_output(True)
    print(s)

### 7、初始化相机并打印检测到的目标看看

In [7]:
#camera = Camera.instance(width=300, height=300)
camera = Camera.instance(width=300, height=300, fps=20)

In [8]:
detections = model(camera.value)
print_names(detections)

### 8、舵机转向的参考值

当目标在摄像头的左侧时，print(center)的值为-0.25左右

当目标在摄像头的右侧时，print(center)的值为0.25左右

当目标在摄像头前方时，print(center)的值为0左右

In [9]:
direction = Pwm()
#direction.switch(direction.turn_left)

# 执行目标检测、目标跟随

In [10]:
blocked_widget = widgets.FloatSlider(min=0.0, max=1.0, value=0.0, description='blocked')
image_widget = widgets.Image(format='jpeg', width=300, height=300)
label_widget = widgets.IntText(value=1, description='tracked label')
speed_widget = widgets.FloatSlider(value=0.4, min=0.0, max=1.0, description='speed')
turn_gain_widget = widgets.FloatSlider(value=0.8, min=0.0, max=2.0, description='turn gain')

display(widgets.VBox([
    widgets.HBox([image_widget, blocked_widget]),
    label_widget,
    speed_widget,
    turn_gain_widget
]))

width = int(image_widget.width)
height = int(image_widget.height)

def detection_center(detection):
    """Computes the center x, y coordinates of the object"""
    bbox = detection['bbox']
    center_x = (bbox[0] + bbox[2]) / 2.0 - 0.5
    center_y = (bbox[1] + bbox[3]) / 2.0 - 0.5
    return (center_x, center_y)
    
def norm(vec):
    """Computes the length of the 2D vector"""
    return np.sqrt(vec[0]**2 + vec[1]**2)

def closest_detection(detections):
    """Finds the detection closest to the image center"""
    closest_detection = None
    for det in detections:
        center = detection_center(det)
        if closest_detection is None:
            closest_detection = det
        elif norm(detection_center(det)) < norm(detection_center(closest_detection)):
            closest_detection = det
    return closest_detection
        
def execute(change):
    global objectId
    
    #这里是没10帧调用1帧
    global t_turn1
    t_turn1 += 1
    if t_turn1 != 5:
        return 0
    else:
        t_turn1 = 0
    
    image = change['new']
    detections = model(image)
    
    print_names(detections)
    
    # 标出所有探测到的对象
    for det in detections[0]:
        bbox = det['bbox']
        cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (255, 0, 0), 2)
    
    # 从对象中找到匹配的类标签
    #matching_detections = [d for d in detections[0] if d['label'] == int(label_widget.value)]
    
    #跟随自己在第一步定义的目标
    matching_detections = [d for d in detections[0] if d['label'] == int(objectId)]
    
    # 标出距视野中心最近的对象
    det = closest_detection(matching_detections)
    if det is not None:
        bbox = det['bbox']
        cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (0, 255, 0), 5)
    
    if det is None:
        robot.forward(0)# 如果没有目标就停止
    else:# 有目标就朝目标走
        # 向前移动小车并根据目标对中心的偏离进行偏转
        center = detection_center(det)
        if center[0] < -0.2:
            direction.switch(direction.turn_left2)
        if center[0] > -0.2 and center[0] < -0.1:
            direction.switch(direction.turn_left)
        if center[0] > 0.1 and center[0] < 0.2:
            direction.switch(direction.turn_right)
        if center[0] > 0.2:
            direction.switch(direction.turn_right2)
        if center[0] > -0.1 and center[0] < 0.1:
            direction.switch(direction.turn_middle)   
        robot.set_motors(130,130)
    
    # 更新图片显示
    image_widget.value = bgr8_to_jpeg(image)
    
execute({'new': camera.value})

0

### 调用相机的图像来检测目标

In [11]:
camera.unobserve_all()

camera.observe(execute, names='value')

看到了物品：cup杯子, 确信度是: 0.4751182198524475



### 停掉小车和相机

In [12]:
camera.unobserve_all()
camera.stop()
time.sleep(1.0)
robot.stop()

# 注意：！！！

### 1、一定要调用camera.stop()停掉相机，否则下次调用相机时很可能会报错

### 2、即使程序执行一半要退出也要camera.stop(),否则再次调用时会报错

### 3、记得用camera.stop()打扫现场